In [ ]:
#read train.json data 
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import json
from pandas.io.json import json_normalize
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline
#read train.json data
train = pd.read_json('train.json')
train.head()
#read test.json data
test = pd.read_json('test.json')
test.head()
